<a href="https://colab.research.google.com/github/mehrshad98/UrbanFloodPredictor/blob/main/ML_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>